In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict,cross_val_score
import matplotlib.pyplot as plt

In [9]:
df = pd.read_csv("/kaggle/input/titanic/train.csv")
df.head(10)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [56]:
#feature engineering
df["Single"]=df['Name'].str.contains(r"Miss")
df["Rich"]=df['Name'].str.contains(r"(Sir\.)(Master\.)")

df.head()

/tmp/ipykernel_33/3440061622.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["Rich"]=df['Name'].str.contains(r"(Sir\.)(Master\.)")


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Single,Rich,T_letter,T_number,T_len
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False,False,A,21171,5.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False,False,PC,17599,5.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,True,False,STON,3101282,7.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False,False,NaN,113803,6.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,False,False,NaN,373450,6.0


In [57]:
df["T_letter"]=df["Ticket"].str.extract(r"([a-zA-Z]+)")
df["T_number"]=df["Ticket"].str.extract(r"(\d+)$")#$ sign ensures take the last digits after the last space
df["T_len"]=df["T_number"].str.len()

In [60]:
df[["c_letter","c_number"]]=df["Cabin"].str.extract(r"([a-zA-Z])(\d+)",expand=True)

In [61]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

cat_pipeline= Pipeline([("impute", SimpleImputer(strategy="constant",fill_value="")),
                       ("encode",OrdinalEncoder())
                       ])

num_pipeline=Pipeline([("impute", SimpleImputer(strategy="median"))
    
])

transformer=ColumnTransformer([
    ("num",num_pipeline,["Pclass","Age","SibSp","Parch","Fare","T_number","c_number","T_len"]),
    ("cat",cat_pipeline,["Sex","Ticket","Cabin","Embarked","Single","Rich","c_letter","T_letter"])
    
])

In [62]:
X=df.iloc[:,2:]
y=df["Survived"]
X_trans=transformer.fit_transform(X,y)

In [63]:
tree = DecisionTreeClassifier(max_depth=5,random_state=1)

In [64]:
cross_val_score(tree,X_trans,y,cv=5,scoring="accuracy").mean()

0.7991463184985249

In [74]:
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(n_estimators=100,criterion="gini",max_samples=0.8)

In [75]:
from sklearn.model_selection import GridSearchCV

grid=GridSearchCV(
    forest,
    {
        "max_depth":[2,4,5,6,7,8,9,10],
        "criterion":["gini","entropy"],
        "ccp_alpha":[0,1,10]#higher alpha you sacrifice the var then prune
        
    },
    cv=5,
    n_jobs=-1
)

In [76]:
grid.fit(X_trans,y)


GridSearchCV(cv=5, estimator=RandomForestClassifier(max_samples=0.8), n_jobs=-1,
             param_grid={'ccp_alpha': [0, 1, 10],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 4, 5, 6, 7, 8, 9, 10]})

In [77]:
grid.best_score_

0.8406565815077522